In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/t-academy-recommendation2/movies/ratings.csv
/kaggle/input/t-academy-recommendation2/movies/movies_metadata.csv
/kaggle/input/t-academy-recommendation2/books/book_tags.csv
/kaggle/input/t-academy-recommendation2/books/ratings.csv
/kaggle/input/t-academy-recommendation2/books/books.csv
/kaggle/input/t-academy-recommendation2/books/sample_book.xml
/kaggle/input/t-academy-recommendation2/books/to_read.csv
/kaggle/input/t-academy-recommendation2/books/train.csv
/kaggle/input/t-academy-recommendation2/books/test.csv
/kaggle/input/t-academy-recommendation2/books/tags.csv
/kaggle/input/contents/movies/movies_metadata.csv
/kaggle/input/contentsbased/movies/movies_metadata.csv


In [2]:
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import os, sys, gc
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5


In [3]:
path = '/kaggle/input/t-academy-recommendation2/books'
print(os.listdir(path))

['book_tags.csv', 'ratings.csv', 'books.csv', 'sample_book.xml', 'to_read.csv', 'train.csv', 'test.csv', 'tags.csv']


In [4]:
books = pd.read_csv('/kaggle/input/t-academy-recommendation2/books/books.csv')
book_tags = pd.read_csv('/kaggle/input/t-academy-recommendation2/books/book_tags.csv')
train = pd.read_csv('/kaggle/input/t-academy-recommendation2/books/train.csv')
test = pd.read_csv('/kaggle/input/t-academy-recommendation2/books/test.csv')
tags = pd.read_csv('/kaggle/input/t-academy-recommendation2/books/tags.csv')
to_read = pd.read_csv('/kaggle/input/t-academy-recommendation2/books/to_read.csv')

In [5]:
train['book_id'] = train['book_id'].astype(str)
test['book_id'] = test['book_id'].astype(str)
books['book_id'] = books['book_id'].astype(str)

In [6]:
popular_rec_model = books.sort_values(by='books_count', ascending=False)['book_id'].values[0:500]

In [7]:
sol = test.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
gt = {}
for user in tqdm(sol['user_id'].unique()):
  gt[user] = list(sol[sol['user_id'] == user]['unique'].values[0])

  0%|          | 0/53424 [00:00<?, ?it/s]

In [8]:
rec_df = pd.DataFrame()
rec_df['user_id'] = train['user_id'].unique()

TF-IDF Contents Based Model

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books['title'])
print(tfidf_matrix.shape)

(10000, 9019)


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_matrix.shape

(10000, 10000)

In [11]:
# Mapping

book2id = {}
for i, c in enumerate(books['title']): book2id[i] = c
    
id2book = {}
for i, c in book2id.items(): id2book[c] = i
    
bookid2book = {}
for i, j in zip(books['title'].values, books['book_id'].values):
    bookid2book[i] = j

In [12]:
books['title'].head()

0              The Hunger Games (The Hunger Games, #1)
1    Harry Potter and the Sorcerer's Stone (Harry P...
2                              Twilight (Twilight, #1)
3                                To Kill a Mockingbird
4                                     The Great Gatsby
Name: title, dtype: object

In [13]:
idx = id2book['Twilight (Twilight, #1)']
sim_scores = [(book2id[i], c) for i, c in enumerate(cosine_matrix[idx]) if i != idx]
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
sim_scores[0:10]

[('The Twilight Saga (Twilight, #1-4)', 0.920347418277986),
 ('The Twilight Collection (Twilight, #1-3)', 0.8786339079447184),
 ('The Twilight Saga Complete Collection  (Twilight, #1-4 + 3.5)',
  0.7697532056304309),
 ('Twilight and History', 0.7465001575650626),
 ('The Twilight Saga: The Official Illustrated Guide (Twilight, #4.5)',
  0.7045174300631831),
 ('Twilight Eyes', 0.6770737331426326),
 ('Twilight (The Mediator, #6)', 0.6377631333498953),
 ('New Moon (Twilight, #2)', 0.6185575138625542),
 ('Eclipse (Twilight, #3)', 0.612819563854136),
 ('The Servants of Twilight', 0.5837817298466093)]

In [14]:
train = pd.merge(train, books[['book_id', 'title']], how='left', on='book_id')
train.head()

,user_id,book_id,title
0,1,4893,NaN
1,2,8855,NaN
2,3,9049,NaN
3,4,3273,Moloka'i
4,5,4829,NaN


In [15]:
# 0. title

tf_train = train[train['title'].notnull()].reset_index(drop=True)
tf_train['idx2title'] = tf_train['title'].apply(lambda x: id2book[x])
tf_train.head()

,user_id,book_id,title,idx2title
0,4,3273,Moloka'i,1215
1,7,4138,Naked,343
2,7,4588,Extremely Loud and Incredibly Close,248
3,9,8676,Unlimited Power : The New Science Of Personal ...,4701
4,10,5907,The Hobbit,6


In [16]:
idx2title2book = {}
for i, j in zip(tf_train['idx2title'].values, tf_train['book_id'].values):
    idx2title2book[i] = j

In [17]:
# 1. User

user = 7
read_list = tf_train.groupby(['user_id'])['idx2title'].agg({'unique'}).reset_index()
seen = read_list[read_list['user_id'] == user]['unique'].values[0]
seen

array([343, 248])

In [18]:
cosine_matrix[343]

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
total_cosine_sim = np.zeros(len(book2id))
for book_ in seen:
    total_cosine_sim += cosine_matrix[book_]

In [20]:
# 3. Score

sim_scores = [(i, c) for i, c in enumerate(total_cosine_sim) if i not in seen]
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
sim_scores[0:5]

[(4809, 0.793036327171204),
 (6199, 0.6915730356677104),
 (3194, 0.6607508855409738),
 (1570, 0.6390974315343301),
 (7393, 0.5820260477746269)]

In [21]:
book2id[4809]

'The Naked and the Dead'

In [22]:
bookid2book[book2id[4809]]

'12467'

In [23]:
tf_train['user_id'].unique()

array([    4,     7,     9, ..., 53416, 53419, 53424])

In [24]:
tf_train.head()

,user_id,book_id,title,idx2title
0,4,3273,Moloka'i,1215
1,7,4138,Naked,343
2,7,4588,Extremely Loud and Incredibly Close,248
3,9,8676,Unlimited Power : The New Science Of Personal ...,4701
4,10,5907,The Hobbit,6


In [25]:
# Total

total_rec_list = {}

read_list1 = train.groupby(['user_id'])['book_id'].agg({'unique'}).reset_index()
read_list2 = tf_train.groupby(['user_id'])['idx2title'].agg({'unique'}).reset_index()

for user in tqdm(train['user_id'].unique()):
    rec_list = []
    
    if user in tf_train['user_id'].unique():
        seen = read_list2[read_list2['user_id'] == user]['unique'].values[0]
        total_cosine_sim = np.zeros(len(book2id))
        for book_ in seen:
            total_cosine_sim += cosine_matrix[book_]
        
        sim_scores = [(bookid2book[book2id[i]], c) for i, c in enumerate(total_cosine_sim) if i not in seen]        
        recs = sorted(sim_scores, key = lambda x: x[1], reverse=True)[0:300]
        
        for rec in recs:
            if rec not in seen:
                      rec_list.append(rec)
     
    else: 
        seen = read_list1[read_list1['user_id'] == user]['unique'].values[0]
        for rec in popular_rec_model[0:400]:
            if rec not in seen:
                rec_list.append(rec)
                
    total_rec_list[user] = rec_list[0:200]  

  0%|          | 0/53382 [00:00<?, ?it/s]

In [26]:
import six
import math

class evaluate():
    def __init__(self, recs, gt, topn=100):
        self.recs = recs
        self.gt = gt 
        self.topn = topn 
        
    def _ndcg(self):
        Q, S = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            dcg = 0.0
            idcg = sum([1.0 / math.log(i + 2, 2) for i in range(min(len(seen), len(rec)))])
            for i, r in enumerate(rec):
                if r not in seen:
                    continue
                rank = i + 1
                dcg += 1.0 / math.log(rank + 1, 2)
            ndcg = dcg / idcg
            S += ndcg
            Q += 1
        return S / Q


    def _map(self):
        n, ap = 0.0, 0.0
        for u, seen in six.iteritems(self.gt):
            seen = list(set(seen))
            rec = self.recs.get(u, [])
            if not rec or len(seen) == 0:
                continue

            _ap, correct = 0.0, 0.0
            for i, r in enumerate(rec):
                if r in seen:
                    correct += 1
                    _ap += (correct / (i + 1.0))
            _ap /= min(len(seen), len(rec))
            ap += _ap
            n += 1.0
        return ap / n


    def _entropy_diversity(self):
        sz = float(len(self.recs)) * self.topn
        freq = {}
        for u, rec in six.iteritems(self.recs):
            for r in rec:
                freq[r] = freq.get(r, 0) + 1
        ent = -sum([v / sz * math.log(v / sz) for v in six.itervalues(freq)])
        return ent
    
    def _evaluate(self):
        print('MAP@%s: %s' % (self.topn, self._map()))
        print('NDCG@%s: %s' % (self.topn, self._ndcg()))
        print('EntDiv@%s: %s' % (self.topn, self._entropy_diversity()))

In [27]:
evaluate_func = evaluate(recs=total_rec_list, gt = gt, topn=200)
evaluate_func._evaluate()

MAP@200: 8.152674144049246e-05
NDCG@200: 0.0008393198172755216
EntDiv@200: 6.916260625417244
